In [1]:
import tensorflow as tf
from ccgnet import experiment as exp
from ccgnet import layers
from ccgnet.layers import *
import numpy as np
import time
from sklearn.metrics import balanced_accuracy_score
from ccgnet.Dataset import Dataset, DataLoader

In [2]:
data = Dataset('./data/CC_Table.tab', mol_blocks_dir='./data/Mol_Blocks.dir')
data.make_graph_dataset(Desc=1, A_type='OnlyCovalentBond', save_name='CC_Dataset_OB.npz')

RDKit ERROR: [16:43:24] UFFTYPER: Unrecognized charge state for atom: 17


Bad input sample:DAQVAS, skipped.


RDKit ERROR: [16:43:51] UFFTYPER: Unrecognized charge state for atom: 0
RDKit WARNING: [16:43:54] UFFTYPER: Warning: hybridization set to SP3 for atom 0


Bad input sample:995&77245, skipped.


RDKit WARNING: [16:44:17] UFFTYPER: Warning: hybridization set to SP3 for atom 0
RDKit WARNING: [16:44:17] UFFTYPER: Warning: hybridization set to SP3 for atom 0
RDKit WARNING: [16:44:18] UFFTYPER: Warning: hybridization set to SP3 for atom 0
RDKit WARNING: [16:44:18] UFFTYPER: Warning: hybridization set to SP3 for atom 0
RDKit WARNING: [16:44:38] UFFTYPER: Warning: hybridization set to SP3 for atom 0
RDKit ERROR: [16:45:08] UFFTYPER: Unrecognized charge state for atom: 0


Bad input sample:NEZROZ, skipped.


RDKit WARNING: [16:45:17] UFFTYPER: Warning: hybridization set to SP3 for atom 0
RDKit ERROR: [16:45:20] UFFTYPER: Unrecognized charge state for atom: 0
RDKit ERROR: [16:45:20] UFFTYPER: Unrecognized charge state for atom: 14
RDKit ERROR: [16:45:44] UFFTYPER: Unrecognized charge state for atom: 0
RDKit WARNING: [16:45:48] UFFTYPER: Warning: hybridization set to SP3 for atom 5
RDKit WARNING: [16:45:54] UFFTYPER: Warning: hybridization set to SP3 for atom 3
RDKit WARNING: [16:45:54] UFFTYPER: Warning: hybridization set to SP3 for atom 3
RDKit ERROR: [16:46:04] UFFTYPER: Unrecognized atom type: S_5+6 (0)


Elapsed Time: 202.75 s


In [3]:
data = DataLoader('CC_Dataset_OB.npz')

In [4]:
from sklearn.model_selection import KFold

test = np.array(eval(open('./data/Test_set.list').read()))
cv = np.array([i for i in data.dataframe.keys() if i not in test])
np.random.shuffle(cv)
fold_10 = {}
fold_10['test'] = test
kf = KFold(n_splits=10, shuffle=True, random_state=1000)
n = 0
for train_idx, test_idx in kf.split(cv):
    fold = 'fold-{}'.format(n)
    fold_10[fold] = {}
    fold_10[fold]['train'] = cv[train_idx]
    fold_10[fold]['valid'] = cv[test_idx]
    n += 1

In [5]:
class CCGNet(object):
    def build_model(self, inputs, is_training, global_step):
        V = inputs[0]
        A = inputs[1]
        labels = inputs[2]
        mask = inputs[3]
        graph_size = inputs[4]
        tags = inputs[5]
        global_state = inputs[6]
        subgraph_size = inputs[7]
        
        global_state = tf.reshape(global_state, [-1, int(global_state.get_shape()[-1].value/2)])
        V, global_state = CCGBlock(V, A, global_state, subgraph_size, no_filters=128, mask=mask, num_updates=global_step, is_training=is_training)
        V, global_state = CCGBlock(V, A, global_state, subgraph_size, no_filters=64, mask=mask, num_updates=global_step, is_training=is_training)
        V, global_state = CCGBlock(V, A, global_state, subgraph_size, no_filters=64, mask=mask, num_updates=global_step, is_training=is_training)
        V, global_state = CCGBlock(V, A, global_state, subgraph_size, no_filters=32, mask=mask, num_updates=global_step, is_training=is_training)
        V, global_state = CCGBlock(V, A, global_state, subgraph_size, no_filters=16, mask=mask, num_updates=global_step, is_training=is_training)
        V = layers.multi_head_global_attention(V, graph_size, num_head=10)
        global_state = tf.reshape(global_state, [-1, 32])
        V = tf.concat([V, global_state], 1)
        V = layers.make_embedding_layer(V, 256, name='FC')
        V = layers.make_bn(V, is_training, mask=None, num_updates=global_step)
        V = tf.nn.relu(V)
        out = layers.make_embedding_layer(V, 2, name='final')
        prob = layers.make_softmax_layer(out, axis=1, name=None)
        return out, labels

In [ ]:
start = time.time()
snapshot_path = './snapshot/'
model_name = 'CCGNet_OB'
dataset_name = 'CB'
History = {}
History['test_acc'] = []
History['val_acc'] = []
for fold in ['fold-{}'.format(i) for i in range(10)]:
    print('\n################ {} ################'.format(fold))
    train_data, valid_data, test_data = data.split(train_samples=fold_10[fold]['train'], test_samples=fold_10[fold]['valid'], val=True, val_samples=fold_10['test'])
    tf.reset_default_graph()
    model = CCGNet()
    model = exp.Model(model, train_data, valid_data, val=True, val_data=test_data, snapshot_path=snapshot_path, use_subgraph=True,
                      model_name=model_name, dataset_name='/time_{}'.format(fold[-1]))
    history = model.fit(save_info=True, save_att=True, silence=0, train_batch_size=256,
                        metric='acc')
    History['test_acc'].append(max(history['test_acc']))
    History['val_acc'].append(history['val_acc'])
print('test_mean_acc:{:.4f}(+-{:.4f})'.format(np.array(History['test_acc']).mean()*100,np.array(History['test_acc']).std()*100))
print('val_mean_acc:{:.4f}(+-{:.4f})'.format(np.array(History['val_acc']).mean()*100,np.array(History['val_acc']).std()*100))
end = time.time()
time_gap = end-start
h = time_gap//3600
h_m = time_gap%3600
m = h_m//60
s = h_m%60
print('{}h {}m {}s'.format(int(h),int(m),round(s,2)))


################ fold-0 ################
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
Creating loss function and summaries
Preparing training
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/export/home/jyy/anaconda3/envs/ccdc/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Starting summaries
## Epoch 0 ==> Train Loss:0.19732, Train Acc:93.12, Test Loss:0.09167, Test Acc:96.88, Elapsed Time:32.31 s
## Epoch 1 ==> Train Loss:0.07867, Train Acc:97.28, Test Loss:0.07416, Test Acc:97.27, Elapsed Time:16.76 s
## Epoch 2 ==> Train Loss:0.06458, Train Acc:97.58, Test Loss:0.08490, Test Acc:96.49, Elapsed Time:15.67 s
## Epoch 3 ==> Train Loss:0.08735, Train Acc:97.19, Test Loss:0.04794, Test Acc:98.31, Elapsed Time:16.37 s
## Epoch 4 ==> Train Loss:0.05823, Train Acc:98.12, Test Loss:0.03860, Test Acc:98.83, Elapsed Time:16.00 s
## Epoch 5 ==> Train Loss:0.04632, Train Acc:98.03, Test Loss:0.04546, Test Acc:98.44, Elapsed Time:15.69 s
## Epoch 6 ==> Train Loss:0.07339, Train Acc:97.39, Test Loss:0.06752, Test Acc:98.31, Elapsed Time:15.81 s
## Epoch 7 ==> Train Loss:0.05109, Train Acc:98.16, Test Loss:0.03601, Test Acc:98.70, Elapsed Time:15.80 s
## Epoch 8 ==> Train Loss:0.04884, Train Acc:98.24, Test Loss:0.05227, Test Acc:97.79, Elapsed Time:15.72 s
## Epoch 

In [10]:
from ccgnet.MetricsReport import model_metrics_report

snapshot_path = './snapshot/'
model_metrics_report('{}/{}/'.format(snapshot_path, model_name))

,Balanced Accuracy,Nacc,Pacc,Precision,Recall,Test Accuracy,Train Accuracy,f1_score
CB,97.7570(±0.9922),95.7230(±1.9480),99.7910(±0.0985),99.3772(±0.2780),99.7910(±0.0985),99.2724(±0.2857),99.2677(±0.4355),99.5835(±0.1623)


In [11]:
from parselog import ParseTestLog, ParseTestLogEnsemble
import glob

val_list = glob.glob('{}/{}/{}/*/*val*'.format(snapshot_path, model_name, 'CB'))
l = []
for i in val_list:
    l.append(ParseTestLog(i))

In [15]:
from parselog import TestAccForEachMol

nico = eval(open('./data/Nicotinamide_Test.list').read())
carb = eval(open('./data/Carbamazepine_Test.list').read())
indo = eval(open('./data/Indomethacin_Test.list').read())
para = eval(open('./data/Paracetamol_Test.list').read())
pyre = eval(open('./data/Pyrene_Test.list.old').read())
print('\n####### Nico ########')
TestAccForEachMol(nico, val_list)
print('\n####### Carb ########')
TestAccForEachMol(carb, val_list)
print('\n####### Indo ########')
TestAccForEachMol(indo, val_list)
print('\n####### Para ########')
TestAccForEachMol(para, val_list)
print('\n####### Pyre ########')
TestAccForEachMol(pyre, val_list)


####### Nico ########
BACC: 95.71(±3.50)
PACC: 100.00(±0.00)
NACC: 91.43(±7.00)
######### Bagging ########
BACC: 96.43
PACC: 100.00
NACC: 92.86

####### Carb ########
BACC: 95.00(±5.45)
PACC: 100.00(±0.00)
NACC: 90.00(±10.90)
######### Bagging ########
BACC: 100.00
PACC: 100.00
NACC: 100.00

####### Indo ########
BACC: 90.59(±4.32)
PACC: 100.00(±0.00)
NACC: 81.18(±8.65)
######### Bagging ########
BACC: 88.24
PACC: 100.00
NACC: 76.47

####### Para ########
BACC: 98.75(±2.50)
PACC: 100.00(±0.00)
NACC: 97.50(±5.00)
######### Bagging ########
BACC: 100.00
PACC: 100.00
NACC: 100.00

####### Pyre ########
BACC: 99.74(±0.40)
PACC: 99.47(±0.80)
NACC: 100.00(±0.00)
######### Bagging ########
BACC: 100.00
PACC: 100.00
NACC: 100.00
